# Create tables for product sales

We start importing the relevant modules and connect to the database callled `sales.db`

In [20]:
import sqlite3
conn = sqlite3.connect("sales.db")
cursor = conn.cursor()

Now we clean the database if there is any already table in there

In [21]:
# Querying for tables in the database
cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
tables = cursor.fetchall()  

# Dropping all tables in the database
if tables:
    print("Tables in the database:")
    for table in tables:
        print(table[0])
        cursor.execute("""DROP TABLE IF EXISTS {}""".format(table[0]))
else:
    print("No tables found in the database.")
print("Database cleared.")

Tables in the database:
products
clients
sales
Database cleared.


## Create table structure

We will create three tables `products` and `clients` which will store information about each entity and the `sales` will estabilish the connection between them.

products: Information about the products. 
* id_product (primary key)
* product_name
* category
* price

sales: Details of each sales transaction.
* id_sale (primary key)
* id_product (foreign key - products table)
* sale_date
* quantity
* id_client (foreign key - clients table)

clients: Informations about clients.
* id_client (primary key)
* client_name
* city
* state

In [22]:
cursor.execute("""CREATE TABLE IF NOT EXISTS products (
    id_product INTEGER PRIMARY KEY,
    name TEXT,
    category INTEGER,
    price REAL
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS clients (
    id_client INTEGER PRIMARY KEY,
    client_name TEXT,
    city TEXT,
    state TEXT
)""")

cursor.execute("""CREATE TABLE IF NOT EXISTS sales (
    id_sale INTEGER PRIMARY KEY,
    id_product INTEGER,
    sale_date TEXT,
    quantity INTEGER,
    id_client INTEGER,
    FOREIGN KEY (id_product) REFERENCES products (id_product),
    FOREIGN KEY (id_client) REFERENCES clients (id_client)
)""")


Check the tables and fields created

In [23]:
# Querying for tables in the database
cursor.execute("""SELECT name FROM sqlite_master WHERE type='table';""")
print( cursor.fetchall() )

cursor.execute("PRAGMA table_info(products)")
print( cursor.fetchall() )

cursor.execute("PRAGMA table_info(clients)")
print( cursor.fetchall() )

cursor.execute("PRAGMA table_info(sales)")
print( cursor.fetchall() )

[('products',), ('clients',), ('sales',)]
[(0, 'id_product', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0), (2, 'category', 'INTEGER', 0, None, 0), (3, 'price', 'REAL', 0, None, 0)]
[(0, 'id_client', 'INTEGER', 0, None, 1), (1, 'client_name', 'TEXT', 0, None, 0), (2, 'city', 'TEXT', 0, None, 0), (3, 'state', 'TEXT', 0, None, 0)]
[(0, 'id_sale', 'INTEGER', 0, None, 1), (1, 'id_product', 'INTEGER', 0, None, 0), (2, 'sale_date', 'TEXT', 0, None, 0), (3, 'quantity', 'INTEGER', 0, None, 0), (4, 'id_client', 'INTEGER', 0, None, 0)]


# Populate the tables

We use the `faker` module to generate synthetic data in a automated manner 

In [24]:
from faker import Faker

fake = Faker('pt_BR')
Faker.seed(1234)


In [25]:
quantity_products = 50
quantity_clients = 100
quantity_sales = 1000

for i in range(quantity_products):
    name = fake.word().capitalize()
    category = fake.random_int(min=1, max=5)
    price = round(fake.random_number(digits=3) + fake.random.random(), 2)
    cursor.execute("""INSERT INTO products (name, category, price) VALUES (?, ?, ?)""", (name, category, price))    
    conn.commit()

for i in range(quantity_clients):
    client_name = fake.name()
    city = fake.city()
    state = fake.state_abbr()
    cursor.execute("""INSERT INTO clients (client_name, city, state) VALUES (?, ?, ?)""", (client_name, city, state))    
    conn.commit()

for i in range(quantity_sales):
    id_product = fake.random_int(min=1, max=50)
    sale_date = fake.date_between(start_date='-1y', end_date='today').strftime('%Y-%m-%d')
    quantity = fake.random_int(min=1, max=10)
    id_client = fake.random_int(min=1, max=100)
    cursor.execute("""INSERT INTO sales (id_product, sale_date, quantity, id_client) VALUES (?, ?, ?, ?)""", (id_product, sale_date, quantity, id_client))    
    conn.commit()

Check the data created by faker

In [26]:
cursor.execute("SELECT * FROM clients LIMIT 10")
cursor.fetchall()

[(1, 'Vitor Borges', 'da Mota das Pedras', 'MG'),
 (2, 'Kamilly Moura', 'Sampaio', 'MG'),
 (3, 'Heitor da Rosa', 'Aragão da Praia', 'TO'),
 (4, 'Isabel Vasconcelos', 'Cavalcante de Garcia', 'MA'),
 (5, 'Brayan Mendes', 'da Rosa das Pedras', 'TO'),
 (6, 'Pietra Cirino', 'Pires do Amparo', 'CE'),
 (7, 'Léo Rezende', 'Costela da Serra', 'PE'),
 (8, 'Natália Rios', 'Ribeiro de Alves', 'SE'),
 (9, 'Sra. Bella Alves', 'Vieira do Amparo', 'AP'),
 (10, 'Lucas Ferreira', 'da Rosa Verde', 'MT')]

In [27]:
cursor.execute("SELECT * FROM products LIMIT 10")
cursor.fetchall()

[(1, 'Nulla', 1, 7.09),
 (2, 'Reprehenderit', 1, 687.69),
 (3, 'Consequuntur', 3, 242.02),
 (4, 'Ad', 3, 663.62),
 (5, 'Sint', 4, 152.09),
 (6, 'Cum', 1, 515.49),
 (7, 'Expedita', 1, 683.54),
 (8, 'Omnis', 1, 615.68),
 (9, 'Quam', 5, 47.27),
 (10, 'Quos', 1, 660.3)]

In [28]:
cursor.execute("SELECT * FROM sales LIMIT 10")
cursor.fetchall()

[(1, 23, '2025-05-14', 1, 65),
 (2, 3, '2024-12-04', 5, 25),
 (3, 31, '2025-07-21', 7, 84),
 (4, 44, '2025-05-29', 5, 41),
 (5, 36, '2025-08-21', 7, 35),
 (6, 29, '2025-05-18', 1, 78),
 (7, 41, '2025-05-06', 5, 25),
 (8, 21, '2024-10-27', 10, 47),
 (9, 24, '2025-02-26', 7, 53),
 (10, 44, '2025-01-04', 10, 22)]